 En este dataset se desea pronosticar el default (pago) del cliente el próximo
 mes a partir de 23 variables explicativas.

   LIMIT_BAL: Monto del credito otorgado. Incluye el credito individual y el credito familiar (suplementario).
   
         SEX: Genero (1=male; 2=female).

   EDUCATION: Educacion (0=N/A; 1=graduate school; 2=university; 3=high school; 4=others).

    MARRIAGE: Estado civil (0=N/A; 1=married; 2=single; 3=others).

         AGE: Edad (years).

       PAY_0: Historia de pagos pasados. Estado del pago en septiembre, 2005.

       PAY_2: Historia de pagos pasados. Estado del pago en agosto, 2005.

       PAY_3: Historia de pagos pasados. Estado del pago en julio, 2005.

       PAY_4: Historia de pagos pasados. Estado del pago en junio, 2005.
       
       PAY_5: Historia de pagos pasados. Estado del pago en mayo, 2005.

       PAY_6: Historia de pagos pasados. Estado del pago en abril, 2005.

   BILL_AMT1: Historia de pagos pasados. Monto a pagar en septiembre, 2005.

   BILL_AMT2: Historia de pagos pasados. Monto a pagar en agosto, 2005.

   BILL_AMT3: Historia de pagos pasados. Monto a pagar en julio, 2005.

   BILL_AMT4: Historia de pagos pasados. Monto a pagar en junio, 2005.

   BILL_AMT5: Historia de pagos pasados. Monto a pagar en mayo, 2005.

   BILL_AMT6: Historia de pagos pasados. Monto a pagar en abril, 2005.

    PAY_AMT1: Historia de pagos pasados. Monto pagado en septiembre, 2005.

    PAY_AMT2: Historia de pagos pasados. Monto pagado en agosto, 2005.

    PAY_AMT3: Historia de pagos pasados. Monto pagado en julio, 2005.

    PAY_AMT4: Historia de pagos pasados. Monto pagado en junio, 2005.

    PAY_AMT5: Historia de pagos pasados. Monto pagado en mayo, 2005.
    
    PAY_AMT6: Historia de pagos pasados. Monto pagado en abril, 2005.

 La variable "default payment next month" corresponde a la variable objetivo.

 El dataset ya se encuentra dividido en conjuntos de entrenamiento y prueba
 en la carpeta "files/input/".

 Los pasos que debe seguir para la construcción de un modelo de
 clasificación están descritos a continuación.

 ## Paso 1.
 Realice la limpieza de los datasets:
 - Renombre la columna "default payment next month" a "default".
 - Remueva la columna "ID".
 - Elimine los registros con informacion no disponible.
 - Para la columna EDUCATION, valores > 4 indican niveles superiores
   de educación, agrupe estos valores en la categoría "others".

 Renombre la columna "default payment next month" a "default"
 y remueva la columna "ID".


 ## Paso 2.
Divida los datasets en x_train, y_train, x_test, y_test.
Separar características (X) y variable objetivo (y)

In [65]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report,precision_score, balanced_accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score,ParameterGrid, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
from itertools import product
import gzip
import joblib
import json
import os
import pickle


def carga_y_limpieza(train_path, test_path):
    # Leer los datasets descomprimidos
    train_dataset = pd.read_csv(train_path)
    test_dataset = pd.read_csv(test_path)

    # --Renombrar columnas
    train_dataset.rename(columns={"default payment next month": "default"}, inplace=True)
    test_dataset.rename(columns={"default payment next month": "default"}, inplace=True)

    # --Remueva la columna "ID".
    train_dataset.drop(columns="ID", inplace=True)
    test_dataset.drop(columns="ID", inplace=True)

    # -- Elimine los registros con informacion no disponible.
    train_dataset.dropna(inplace=True)
    test_dataset.dropna(inplace=True)

    # - Para la columna EDUCATION, valores > 4 indican niveles superiores
    #   de educación, agrupe estos valores en la categoría "others".
    train_dataset.loc[train_dataset["EDUCATION"] > 4, "EDUCATION"] = 5  # Usar 5 para "others"
    test_dataset.loc[test_dataset["EDUCATION"] > 4, "EDUCATION"] = 5

    x_train = train_dataset.drop(columns=["default"])
    y_train = train_dataset["default"]

    # Características y variable objetivo para prueba
    x_test = test_dataset.drop(columns=["default"])
    y_test = test_dataset["default"]
    categorical_columns = ["SEX", "EDUCATION", "MARRIAGE"]
    
    for col in categorical_columns:
        x_train[col] = x_train[col].astype(int)
        x_test[col] = x_test[col].astype(int)
    # --Identificar columnas numéricas (todas excepto las categóricas)
    numeric_columns = [col for col in x_train.columns if col not in categorical_columns]

    return x_test, x_train, y_test, y_train, categorical_columns, numeric_columns

 ## Paso 3.
 Cree un pipeline para el modelo de clasificación. Este pipeline debe
 contener las siguientes capas:
 - Transforma las variables categoricas usando el método
   one-hot-encoding.
 - Escala las demas variables al intervalo [0, 1].
 - Selecciona las K mejores caracteristicas.
 - Ajusta un modelo de regresion logistica.

In [66]:
def build_pipeline(estimator, categorical_columns, numeric_columns):

    from sklearn.compose import ColumnTransformer
    from sklearn.feature_selection import SelectKBest, f_classif
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import MinMaxScaler

    
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_columns),
            ("num", MinMaxScaler(), numeric_columns),
        ],
        remainder="drop"  # ya estás especificando todas las columnas necesarias
    )
    selectkbest = SelectKBest(score_func=f_classif)

    pipeline = Pipeline(
        steps=[
            ("tranformer", preprocessor),
            ("selectkbest", selectkbest),
            ("estimator", estimator),
        ],
        verbose=False,
    )

    return pipeline  

## Paso 4.
 Optimice los hiperparametros del pipeline usando validación cruzada.
 Use 10 splits para la validación cruzada. Use la función de precision
 balanceada para medir la precisión del modelo.
## Paso 5.
 Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
 Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [67]:
def make_grid_search(estimator, param_grid, cv=10):

    from sklearn.model_selection import GridSearchCV

    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=cv,
        scoring="balanced_accuracy",
        verbose= 2
    )

    return grid_search  

In [68]:
def save_estimator(estimator):
    import gzip
    import joblib
    import os

    os.makedirs("files/models", exist_ok=True)
    with gzip.open("files/models/model.pkl.gz", "wb") as f:
        pickle.dump(estimator, f)


def load_estimator():
    import gzip
    import joblib
    import os

    path = "files/models/model.pkl.gz"
    if not os.path.exists(path):
        return None

    with gzip.open(path, "rb") as f:
        estimator = pickle.load(f)
    return estimator

In [69]:
def train_estimator(estimator, x_train, y_train, x_test, y_test):
    from sklearn.metrics import balanced_accuracy_score
    import gzip
    import joblib
    import os

    # --- Entrenar el modelo ---
    estimator.fit(x_train, y_train)

    # --- Obtener el mejor modelo (del grid search) ---
    best_model = estimator.best_estimator_

    # --- Evaluar ---
    y_pred = best_model.predict(x_test)
    score = balanced_accuracy_score(y_test, y_pred)

    # --- Comparar con modelo guardado ---
    best_saved_model = load_estimator()

    if best_saved_model is not None:
        y_pred_saved = best_saved_model.predict(x_test)
        saved_score = balanced_accuracy_score(y_test, y_pred_saved)

        # Si el modelo guardado es mejor, lo conservamos
        if saved_score > score:
            print("El modelo anterior tiene mejor desempeño. Se conserva el guardado.")
            return best_saved_model

    # --- Guardar el nuevo modelo ---
    save_estimator(best_model)
    print(f"Nuevo modelo guardado con balanced_accuracy: {score:.4f}")
    return best_model

In [70]:
def train_logistic_regression(train_path, test_path):
    from sklearn.linear_model import LogisticRegression

    # --- Paso 1 y 2: Carga y limpieza ---
    x_test, x_train, y_test, y_train, categorical_columns, numeric_columns = carga_y_limpieza(
        train_path, test_path
    )

    # --- Paso 3: Pipeline ---
    pipeline = build_pipeline(
        estimator=LogisticRegression(max_iter=10000, solver="saga"),
        categorical_columns=categorical_columns,
        numeric_columns=numeric_columns,
    )

    # --- Paso 4: Grid search ---
    param_grid = {
    "selectkbest__k": [1, 3, 5],
    "estimator__C": [0.5, 1, 2, 3, 5],
    "estimator__penalty": ["l1", "l2"],
    "estimator__class_weight": [None, "balanced"]
}

    grid_search = make_grid_search(
        estimator=pipeline,
        param_grid=param_grid,
        cv=10,
    )

    # Entrenar y guardar el mejor modelo
    train_estimator(grid_search, x_train, y_train, x_test, y_test)

 ## Paso 6.
 Calcule las metricas de precision, precision balanceada, recall,
 y f1-score para los conjuntos de entrenamiento y prueba.
 Guardelas en el archivo files/output/metrics.json. Cada fila
 del archivo es un diccionario con las metricas de un modelo.
 Este diccionario tiene un campo para indicar si es el conjunto
 de entrenamiento o prueba. Por ejemplo:

 {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
 {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

 ## Paso 7.
 Calcule las matrices de confusion para los conjuntos de entrenamiento y
 prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
 del archivo es un diccionario con las metricas de un modelo.
 de entrenamiento o prueba. Por ejemplo:

 {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
 {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}




In [71]:
def evaluate_and_save_metrics(model, X_train, y_train, X_test, y_test, output_path):
    """
    Calcula métricas (precision, recall, f1, balanced_accuracy) y matrices de confusión
    para train/test, y las guarda en formato JSONL con la estructura requerida.
    """
    import os, json
    from sklearn.metrics import (
        precision_score,
        recall_score,
        f1_score,
        balanced_accuracy_score,
        confusion_matrix,
    )

    results = []

    # === Métricas para train y test ===
    for dataset, X, y in [("train", X_train, y_train), ("test", X_test, y_test)]:
        y_pred = model.predict(X)

        precision = precision_score(y, y_pred, zero_division=0)
        recall = recall_score(y, y_pred, zero_division=0)
        f1 = f1_score(y, y_pred, zero_division=0)
        balanced_acc = balanced_accuracy_score(y, y_pred)

        metrics_dict = {
            "type": "metrics",
            "dataset": dataset,
            "precision": round(precision, 3),
            "balanced_accuracy": round(balanced_acc, 3),
            "recall": round(recall, 3),
            "f1_score": round(f1, 3),
        }
        results.append(metrics_dict)

    # === Matrices de confusión ===
    for dataset, X, y in [("train", X_train, y_train), ("test", X_test, y_test)]:
        y_pred = model.predict(X)
        cm = confusion_matrix(y, y_pred)

        # Asegurar formato 2x2
        if cm.shape == (2, 2):
            cm_dict = {
                "type": "cm_matrix",
                "dataset": dataset,
                "true_0": {"predicted_0": int(cm[0, 0]), "predicted_1": int(cm[0, 1])},
                "true_1": {"predicted_0": int(cm[1, 0]), "predicted_1": int(cm[1, 1])},
            }
        else:
            # Por si hay clases ausentes (modelo predijo solo una clase)
            cm_dict = {
                "type": "cm_matrix",
                "dataset": dataset,
                "true_0": {"predicted_0": None, "predicted_1": None},
                "true_1": {"predicted_0": None, "predicted_1": None},
            }

        results.append(cm_dict)

    # === Guardar todo en JSONL ===
# Crear carpeta si no existe (maneja el caso donde no hay carpeta en output_path)
    directory = os.path.dirname(output_path)
    if directory:  # Solo crea si hay una carpeta especificada
        os.makedirs(directory, exist_ok=True)

    with open(output_path, "a") as f:
        for r in results:
            f.write(json.dumps(r) + "\n")

    print(f"✅ Resultados guardados en {output_path}")

    print(f"✅ Métricas y matrices guardadas en {output_path}")

## Ejecución del flujo

In [72]:
def train_and_evaluate_logistic_regression():
    import os
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import make_pipeline
    from sklearn.feature_selection import SelectKBest, f_classif

    # === 1. Cargar datos ===
    
    train_path="/Users/sebastiangomezzapata/Universidad/Semestre 2025-II/Analítica predictiva/LAB-02-prediccion-del-default-usando-logreg-segomezz/files/input/train_default_of_credit_card_clients.csv"
    test_path="/Users/sebastiangomezzapata/Universidad/Semestre 2025-II/Analítica predictiva/LAB-02-prediccion-del-default-usando-logreg-segomezz/files/input/test_default_of_credit_card_clients.csv"
    X_train, X_test, y_train, y_test,categorical_columns, numeric_columns = carga_y_limpieza(train_path,test_path)
    

    # === 2. Crear pipeline ===
    pipeline = build_pipeline(
        estimator=LogisticRegression(max_iter=10000, solver="saga"),
        categorical_columns=categorical_columns,
        numeric_columns=numeric_columns
    )

    # === 3. Definir espacio de búsqueda ===
    param_grid = {
    "selectkbest__k": [3, 5, 8, 10],
    "estimator__C": [1, 2, 3, 5],
    "estimator__penalty": ["l1", "l2"],
    "estimator__class_weight": [None, "balanced"],
    "estimator__solver": ["liblinear", "saga"]
}
    # === 4. Crear y entrenar grid search ===
    grid_search = make_grid_search(
        estimator=pipeline,
        param_grid=param_grid,
        cv=10,
    )
    

    print("🚀 Entrenando modelo con GridSearchCV...")
    grid_search.fit(X_train, y_train)
    # === 6. Diagnóstico ===
    print("➡️ Best estimator:", grid_search.best_estimator_)
    print("➡️ Best params:", grid_search.best_params_)
    print("➡️ Classes:", grid_search.best_estimator_.named_steps['estimator'].classes_)

    # === 5. Guardar el mejor modelo ===
    best_model = grid_search
    save_estimator(best_model)
    print("✅ Mejor modelo guardado en 'estimator.pickle'")

    # === 6. Calcular y guardar métricas ===
    output_path = "files/output/metrics.json"
    evaluate_and_save_metrics(best_model, X_train, y_train, X_test, y_test, output_path)

    # === 7. Mostrar mejores hiperparámetros ===
    print("\n🏆 Mejores hiperparámetros encontrados:")
    print(grid_search.best_params_)

    print("\n✅ Entrenamiento y evaluación completados.")


# Ejecutar todo el flujo
train_and_evaluate_logistic_regression()

🚀 Entrenando modelo con GridSearchCV...
Fitting 10 folds for each of 64 candidates, totalling 640 fits
[CV] END estimator__C=0.1, estimator__class_weight=None, estimator__penalty=l1, estimator__solver=liblinear, selectkbest__k=3; total time=   0.0s
[CV] END estimator__C=0.1, estimator__class_weight=None, estimator__penalty=l1, estimator__solver=liblinear, selectkbest__k=3; total time=   0.0s
[CV] END estimator__C=0.1, estimator__class_weight=None, estimator__penalty=l1, estimator__solver=liblinear, selectkbest__k=3; total time=   0.0s
[CV] END estimator__C=0.1, estimator__class_weight=None, estimator__penalty=l1, estimator__solver=liblinear, selectkbest__k=3; total time=   0.0s
[CV] END estimator__C=0.1, estimator__class_weight=None, estimator__penalty=l1, estimator__solver=liblinear, selectkbest__k=3; total time=   0.0s
[CV] END estimator__C=0.1, estimator__class_weight=None, estimator__penalty=l1, estimator__solver=liblinear, selectkbest__k=3; total time=   0.0s
[CV] END estimator__C